In [125]:
import pandas as pd
from geopy.geocoders import Nominatim

chunksize = 10 ** 4  # adjust this value depending on your available memory
chunks = []

selected_features = [
    "COUNTY",
    "total_homes_sold",
    "median_sale_price",
    "median_sale_ppsf",
    "price_drops",
    "percent_active_listings_with_price_drops",
    "median_days_to_close",
    "pending_sales",
    "inventory",
    "median_active_list_price",
    "median_active_list_ppsf",
    "median_days_on_market",
    "months_of_supply",
    "total_homes_sold_with_price_drops",
    "percent_homes_sold_with_price_drops",
    "average_of_median_offer_price_amount",
    "average_sale_to_list_ratio",
    "percent_homes_sold_above_list",
    "age_of_inventory",
    "median_pending_sqft",
    #"month" Assuming you've created this feature from "period_begin" or "period_end"
]

for chunk in pd.read_csv('housingdata.tsv', delimiter='\t', chunksize=chunksize):
    # process the chunk, for example, filter rows or columns
    filtered_chunk = chunk[selected_features]
    chunks.append(filtered_chunk)

# Concatenate all chunks into one DataFrame
df = pd.concat(chunks, axis=0)

df = df.dropna()

print(df.head())



    COUNTY  total_homes_sold  median_sale_price  median_sale_ppsf  \
4     1315              44.0           283500.0        161.276224   
5     1315             471.0           283799.7        152.768580   
6     1315             180.0           286875.0        157.181841   
49    1315              79.0           287000.0        151.975684   
50    1315             510.0           282091.6        153.066718   

    price_drops  percent_active_listings_with_price_drops  \
4          19.0                                  0.027182   
5          13.0                                  0.020966   
6          17.0                                  0.025415   
49         19.0                                  0.026135   
50         14.0                                  0.021709   

    median_days_to_close  pending_sales  inventory  median_active_list_price  \
4                   44.5           46.0      639.0                  350000.0   
5                   41.5           40.0      591.0        

In [129]:

selected_features1 = [
    'COUNTY',
    'median_sale_price',
    'median_sale_ppsf',
    'total_homes_sold',
    'percent_homes_sold_with_price_drops',
    'price_drops',
    'inventory',
    'total_active_listings',
    'median_active_list_price',
    #"month" Assuming you've created this feature from "period_begin" or "period_end"
]

selected_features2 = [
    'PROPERTY TYPE',
    'CITY',
    'ZIP OR POSTAL CODE',
    'BEDS',
    'BATHS',
    'SQUARE FEET',
    'LOT SIZE',
    'YEAR BUILT',
    'HOA/MONTH',
    'LATITUDE',
    'LONGITUDE',
    'PRICE',
    'DAYS ON MARKET',
]


file1 = 'housingdata.tsv'  # Macro-level real estate data
file2 = 'housingdata2.csv'  # Transaction-level real estate data
geolocator = Nominatim(user_agent='my-real-estate-application321')
chunk_size = 10000  # Adjust the chunk size as per your requirements

# Load the first dataset (macro-level real estate data)
df1_chunks = pd.read_csv(file1, sep='\t', chunksize=chunk_size)
df1_list = []

for chunk in df1_chunks:
    # Process each chunk (selecting desired features, renaming columns, etc.)
    chunk_selected = chunk[selected_features1]
    # Append the processed chunk to the list
    df1_list.append(chunk_selected)

# Concatenate all the processed chunks into a single DataFrame
df1 = pd.concat(df1_list)

print(len(df1))
df1_combined = df1.groupby('COUNTY').agg({
    'median_sale_price': 'mean',
    'median_sale_ppsf': 'mean',
    'total_homes_sold': 'sum',
    'percent_homes_sold_with_price_drops': 'mean',
    'price_drops': 'sum',
    'inventory': 'mean',
    'total_active_listings': 'sum',
    'median_active_list_price': 'mean'
}).reset_index()

print(len(df1_combined))
print(df1.head())


3646956


KeyError: 'region_name'

In [97]:

# Load the second dataset (real estate transactions)

merged_data = pd.DataFrame()

df2_chunks = pd.read_csv(file2, sep=',', chunksize=chunk_size)
df2_list = []

for chunk in df2_chunks:
    print(chunk)

                 SALE TYPE  SOLD DATE              PROPERTY TYPE  \
0              MLS Listing        NaN  Single Family Residential   
1              MLS Listing        NaN  Single Family Residential   
2              MLS Listing        NaN                  Townhouse   
3              MLS Listing        NaN  Single Family Residential   
4              MLS Listing        NaN  Single Family Residential   
..                     ...        ...                        ...   
130  New Construction Plan        NaN  Single Family Residential   
131  New Construction Plan        NaN  Single Family Residential   
132  New Construction Plan        NaN  Single Family Residential   
133  New Construction Plan        NaN  Single Family Residential   
134  New Construction Plan        NaN  Single Family Residential   

                ADDRESS      CITY STATE OR PROVINCE  ZIP OR POSTAL CODE  \
0       65 Lafayette St  Stafford                VA               22554   
1        316 Decatur Rd  Stafford

In [119]:
df2 = pd.read_csv(file2, sep=',')
df2_list = []
df2['COUNTY'] = df2.apply(lambda row: geolocator.reverse((row['LATITUDE'], row['LONGITUDE'])).raw['address'].get('county').upper(), axis=1)



In [128]:
df1_combined.set_index('COUNTY', inplace=True)
df2.set_index('COUNTY', inplace=True)

# Join the datasets based on the index (COUNTY)
merged_data = df2.join(df1, how='left')

# Reset the index to make COUNTY a regular column again
merged_data.reset_index(inplace=True)

# Print the merged dataset
print(merged_data.head())

KeyError: "None of ['COUNTY'] are in the columns"